### Environment Loading

- `dotenv`is used to load environment variables from a `.env` file.
- After `load_dotenv()` is used to fetch the environment keys 
- If api key is not found, it raises a `ValueError`.

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv() # load environment variables from .env file

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY in .env")

### LangChain Chat Model Setup

- `ChatOpenAI` is LangChain's wrapper around OpenAI's chat endpoints
- `model="gpt-4o-mini"` specifies the model to use
- `temperature=0.7` controls the randomness of the model's responses
- `openai_api_key=OPENAI_API_KEY` sets the API key for authentication

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.7,
    openai_api_key = OPENAI_API_KEY
)

### Defining a Custom Tool

- The `@tool` decorator registers Python function so LangChain can call it.
- `simple_math_tool` takes a string `("5+7")` and returns the computed result
- Wrapping it in `Tool(...)` gives:
    - A name `("Calculator")`
    - The actual **callable** `(func=…)`
    - A description so agents know when to use it.

In [18]:
from langchain.tools import Tool
from langchain.agents.tools import tool

# @tool(description="Do basic math from string input like '2 + 2'.")
@tool
def simple_math_tool(query: str) -> str:
    """Do basic math from string input like '2 + 2'."""     # LLM needs a tool description to use it correctly
    try:
        return f"Math result: {eval(query)}"
    except Exception as e:
        return f"Error: {e}"

math_tool = Tool(
    name="Calculator",
    func=simple_math_tool,
    description="Do basic math"
)


### Creating a LangChain Agent

- Agents are “smart” wrappers that decide which tool (or LLM) to call given a user query.

- `ZERO_SHOT_REACT_DESCRIPTION` means:

    - Read the user’s input.

    - “Reason” (REACT) about which tool to use (zero‑shot, no prior examples).

- `tools=[math_tool]` tells it you only have a calculator available.

- `verbose=True` prints out what the agent is doing step by step.

In [20]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent = initialize_agent(
    tools=[math_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

C:\Users\HirudikaAnupama\AppData\Local\Temp\ipykernel_15524\125389324.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### Defining LangGraph Nodes

- Nodes are pure Python functions that take a state dict and return a dict (new state) or a branch label.

- `greet_node`

    - Prints a greeting.

    - Returns `{"input": inp}` so the user text is carried forward.

- `router_fn`

    - Reads `state["input"].`

    - If it sees a math operator, returns the string `"math"`, else `"chat".`

    - That string tells LangGraph which node to execute next.

- `math_node`

    - Calls the LangChain agent, which in turn calls your `Calculator` tool.

    - Returns `{"response": ...},` storing the answer in state.

- `chat_node`

    - Calls the LLM directly via `llm.invoke(...).`

    - Returns the LLM’s answer in `state["response"].`

In [21]:
from langgraph.graph import StateGraph, END

def greet_node(state):
    inp = state.get("input", "")
    print("Greet Node")
    print("Bot: Hi! Ask me anything.")
    return {"input": inp}

In [22]:
def router_fn(state) -> str:
    inp = state["input"]
    print("Routing based on input...")
    return "math" if any(op in inp for op in ["+", "-", "*", "/", "%", "**"]) else "chat"

In [24]:
def math_node(state):
    inp = state["input"]
    print("Math Node")
    resp = agent.run(inp)
    return {"input": inp, "response": resp}

In [25]:
def chat_node(state):
    inp = state["input"]
    print("💬 Chat Node")
    resp = llm.invoke(inp)
    return {"input": inp, "response": resp.content}

### Building & Compiling the StateGraph

- `StateGraph(dict)`: uses a simple Python dict for state storage.

- `add_node(name, fn)` registers each node under a key.

- `set_entry_point("greet")` says the first node to run is greet.

- `add_conditional_edges("greet", router_fn, {...})`:

    - After `greet_node`, call `router_fn(state).`

    - If it returns `"math"`, jump to `math` node; if "chat", go to chat.

- `add_edge("math", END)` and `("chat", END)` end the workflow and return the final state.

- `.compile()` optimizes the graph into an executable engine `(app).`

In [26]:
graph = StateGraph(dict)

graph.add_node("greet", greet_node)
graph.add_node("math",   math_node)
graph.add_node("chat",   chat_node)

graph.set_entry_point("greet")

graph.add_conditional_edges(
    "greet",
    router_fn,
    {"math": "math", "chat": "chat"}
)

graph.add_edge("math", END)
graph.add_edge("chat", END)

app = graph.compile()


### The Chat Loop

In [ ]:
if __name__ == "__main__":
    while True:
        q = input("\n You: ")
        if q.strip().lower() in {"exit", "quit", "bye"}:
            print(" Goodbye!")
            break
        out = app.invoke({"input": q})
        print(" Bot:", out["response"])


Greet Node
Bot: Hi! Ask me anything.
Routing based on input...
💬 Chat Node
 Bot: Hello! How can I assist you today?
Greet Node
Bot: Hi! Ask me anything.
Routing based on input...
Math Node


> Entering new AgentExecutor chain...


C:\Users\HirudikaAnupama\AppData\Local\Temp\ipykernel_15524\503184002.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp = agent.run(inp)


I need to perform a basic addition operation to find the sum of 2 and 2.  
Action: Calculator  
Action Input: '2 + 2'  
Observation: Math result: 2 + 2
Thought:The addition of 2 and 2 results in 4.  
Action: Calculator  
Action Input: '2 + 2'  
Observation: Math result: 2 + 2
Thought:The addition of 2 and 2 results in 4.  
Final Answer: 4

> Finished chain.
 Bot: 4
Greet Node
Bot: Hi! Ask me anything.
Routing based on input...
💬 Chat Node
 Bot: It seems like you might have meant "Elon Musk." He is a prominent entrepreneur and business magnate known for founding and leading several high-profile technology companies. Elon Musk is the CEO and lead designer of SpaceX, CEO and product architect of Tesla, Inc., and has been involved in various other ventures such as Neuralink and The Boring Company. He is also one of the co-founders of PayPal. Musk is known for his ambitious goals, including advancing space exploration, promoting sustainable energy, and developing technologies for brain-comp